<a href="https://colab.research.google.com/github/MilciadesGonzalez/TradeLaVirginia/blob/main/Precios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
hecha = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/1.6 - Correción de Precios/Precio con Python/DescargaEncuestaPrecios.xlsx')
objBocas = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/1.6 - Correción de Precios/Precio con Python/Obj_Bocas.xlsx')
objMaterial = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/1.6 - Correción de Precios/Precio con Python/Obj_Producto.xlsx')
regional = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/1.6 - Correción de Precios/Precio con Python/Region.xlsx')

In [ ]:
cierreSemanal = '2023-10-09'
cierreQuincenal = '2023-10-02'
cierreG1 = '2023-09-18'
cierreG2 = '2023-10-02'
cierreSemanal = pd.to_datetime(cierreSemanal)
cierreQuincenal = pd.to_datetime(cierreQuincenal)
cierreG1 = pd.to_datetime(cierreG1)
cierreG2 = pd.to_datetime(cierreG2)

In [ ]:
obj_hechas = hecha
aux_hechas = pd.merge(objBocas, hecha, on=['CódigoBoca', 'Relevamiento'], how='left')
noHechas = aux_hechas[aux_hechas['Relevado'].isnull()]
hechas = obj_hechas[obj_hechas['Relevado'].notnull()]
noHechas = noHechas.rename(columns={'Sucursal_x':'Sucursal', 'Cliente_x':'Cliente','Boca_x':'Boca','Categoria_x':'Categoria','Repositor_x':'Repositor','Supervisor_x':'Supervisor'})
noHechas = noHechas[['Sucursal','Cliente', 'CódigoBoca', 'Boca','Categoria', 'Relevamiento', 'Relevado', 'Repositor', 'Supervisor','CodigoProducto','Producto']]
hechas = hechas.rename(columns={'Sucursal_y':'Sucursal', 'Cliente_y':'Cliente','Boca_y':'Boca','Categoria_y':'Categoria','Repositor_y':'Repositor','Supervisor_y':'Supervisor'})
hechas = hechas[['Sucursal','Cliente', 'CódigoBoca', 'Boca','Categoria', 'Relevamiento', 'Relevado', 'Repositor', 'Supervisor','CodigoProducto','Producto']]

In [ ]:
obj_material = pd.merge(objMaterial, hecha, on=['CodigoProducto', 'Relevamiento'], how='left')
matHechas = obj_material[obj_material['Producto_y'].notnull()]
matNoHechas = obj_material[obj_material['Producto_y'].isnull()]
matHechas = matHechas.rename(columns={'Producto_x':'Producto'})
matNoHechas = matNoHechas.rename(columns={'Producto_x':'Producto'})
matHechas = matHechas[['Sucursal','Cliente', 'CódigoBoca', 'Boca','Categoria', 'Relevamiento', 'Relevado', 'Repositor', 'Supervisor','CodigoProducto','Producto']]
matNoHechas = matNoHechas[['Relevamiento', 'CodigoProducto','Producto']]
matHechas = matHechas.drop_duplicates(subset=['CodigoProducto','CódigoBoca','Relevamiento'])
matNoHechas = matNoHechas.drop_duplicates(subset=['CodigoProducto','Relevamiento'])

In [ ]:
matNoHechas.to_excel('matNoHechas_.xlsx', index=False)
!cp matNoHechas_.xlsx "/content/drive/Shareddrives/Trade Mktg/Trade/1.6 - Correción de Precios/Precio con Python/"

In [ ]:
td_hecha = matHechas.pivot_table(index=('CódigoBoca', 'Relevamiento'), values='CodigoProducto', aggfunc='count')
td_objMat = objMaterial.pivot_table(index='Relevamiento', values='CodigoProducto', aggfunc='count')
td_hecha =  td_hecha.reset_index()
td_objMat =  td_objMat.reset_index()

In [ ]:
noHechas = pd.merge(noHechas, hechas, on='CódigoBoca', how='left')
noHechas = noHechas.rename(columns={'Sucursal_x':'Sucursal', 'Cliente_x':'Cliente', 'Boca_x':'Boca', 'Categoria_x':'Categoria','Relevamiento_x':'Relevamiento','Repositor_x':'Repositor',
                                'Supervisor_x':'Supervisor'})
noHechas['Repositor'] = noHechas['Repositor'].fillna(noHechas['Repositor'])

In [ ]:
noHechas['Relevado'] = ''

In [ ]:
hechas = hechas[['Sucursal', 'Cliente', 'CódigoBoca', 'Boca', 'Categoria', 'Relevamiento', 'Relevado', 'Repositor', 'Supervisor']]
noHechas = noHechas[['Sucursal', 'Cliente', 'CódigoBoca', 'Boca', 'Categoria', 'Relevamiento', 'Relevado', 'Repositor', 'Supervisor']]
hechas['Status'] = 'Hecha'
noHechas['Status'] = 'NoHecha'
hechas = hechas.drop_duplicates(subset=['CódigoBoca','Relevamiento'])
hechas = pd.merge(hechas, td_hecha, on=['CódigoBoca','Relevamiento'], how='left')
hechas = pd.merge(hechas, td_objMat, on='Relevamiento', how='left')
noHechas = pd.merge(noHechas, td_objMat, on='Relevamiento', how='left')
noHechas['CodigoProducto_x'] = 0
hechas = hechas.rename(columns={'CodigoProducto_x':'ProductosRelevados', 'CodigoProducto_y':'ObjProductos'})
noHechas = noHechas.rename(columns={'CodigoProducto_x':'ProductosRelevados', 'CodigoProducto':'ObjProductos'})

<ipython-input-10-c78a89473186>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hechas['Status'] = 'Hecha'


In [ ]:
base = pd.concat([hechas, noHechas])
base['PreciosTomados'] = np.where((base.Status == 'Hecha'), 1, 0)
base['PreciosSinTomar'] = np.where((base.Status == 'NoHecha'), 1, 0)
base['TotalGral'] = 1
base = pd.merge(base, regional, on='Sucursal', how='left')
base['CódigoBoca'] = '0' + base['CódigoBoca'].astype(str)
base = base.drop_duplicates(subset=['CódigoBoca','Relevamiento'])

In [ ]:
base['Relevado'] = np.where(((base.Relevamiento == 'Cadenas Semanal')|(base.Relevamiento == 'Mayoristas Semanal')),cierreSemanal,
                                np.where(((base.Relevamiento == 'Cadenas Mensual Grupo 1')|(base.Relevamiento == 'Mayoristas Mensual Grupo 1')),cierreG1,
                                np.where(((base.Relevamiento == 'Cadenas Mensual Grupo 2')|(base.Relevamiento == 'Mayoristas Mensual Grupo 2')),cierreG2,
                                np.where(((base.Relevamiento == 'Autoservicios Quincenal (V1)')|(base.Relevamiento == 'Minoristas Quincenal (V1)')),cierreQuincenal,
                                np.where(((base.Relevamiento == 'Autoservicios Mensual Grupo 1')|(base.Relevamiento == 'Minoristas Mensual Grupo 1')),cierreG1,
                                np.where(((base.Relevamiento == 'Autoservicios Mensual Grupo 2')|(base.Relevamiento == 'Minoristas Mensual Grupo 2')),cierreG2,''))))))

<ipython-input-12-a0b4d1d4c24d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base['Relevado'] = np.where(((base.Relevamiento == 'Cadenas Semanal')|(base.Relevamiento == 'Mayoristas Semanal')),cierreSemanal,


In [ ]:
base.to_excel('Base_.xlsx', index=False)
!cp Base_.xlsx "/content/drive/Shareddrives/Trade Mktg/Trade/1.6 - Correción de Precios/Precio con Python/"